In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [2]:
import numpy 
import tflearn
import tensorflow 
import random
import json
import pickle

/Users/Eric/opt/anaconda3/envs/smithbot/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/Eric/opt/anaconda3/envs/smithbot/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/Eric/opt/anaconda3/envs/smithbot/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/Eric/opt/anaconda3/envs/

In [3]:
with open("intents.json") as file:
    data = json.load(file)

Next is the words preprossing 

In [4]:
try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words =[]
    labels = []
    docs_x = []
    docs_y = []

    for intents in data["intents"]:
        for patterns in intents["patterns"]:
            wrds = nltk.word_tokenize(patterns)
            words.extend(wrds)
            docs_x.append(wrds)#this gets out the words with all the different patterns 
            docs_y.append(intents["tag" ])

            if intents["tag"] not in labels:
                labels.append(intents["tag"]) 
    words = [stemmer.stem(w.lower()) for w in words]
    words = sorted(list(set(words)))
    labels = sorted(labels)
    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []
        wrds = [stemmer.stem(w) for w in doc if w not in "?"]
    
    for w in words:
        if w in wrds: 
            bag.append(1)
        else:
            bag.append(0)
    
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    
    training.append(bag)
    output.append(output_row)
    
    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []
        wrds = [stemmer.stem(w) for w in doc if w not in "?"]
    
        for w in words:
            if w in wrds: 
                bag.append(1)
        else:
            bag.append(0)
    
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    
    training.append(bag)
    output.append(output_row)
    
    training = numpy.array(training)
    output = numpy.array(output)
    
    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f) 

now we will start creating our trainning and testing output, right now we have strings and neural networks only understand numbers 

In [5]:
#training = []
#output = []

#out_empty = [0 for _ in range(len(labels))]

#for x, doc in enumerate(docs_x):
 #   bag = []
  #  wrds = [stemmer.stem(w) for w in doc if w not in "?"]
    
   # for w in words:
    #    if w in wrds: 
     #       bag.append(1)
      #  else:
       #     bag.append(0)
    
    #output_row = out_empty[:]
    #output_row[labels.index(docs_y[x])] = 1
    
    #training.append(bag)
    #output.append(output_row)
    

In [6]:
#training = numpy.array(training)
#output = numpy.array(output)

In [7]:
tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])]) #this defines the input shape for the model 
net = tflearn.fully_connected(net, 8) #this specifies that we have 8 hidden layers for the hidden layer in the neural network 
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
keep_dims is deprecated, use keepdims instead


Tranning the model 

In [8]:
model = tflearn.DNN(net)

try:
    model.load("model.tflearn.chatbot")
except:
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn.chatbot")


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /Users/Eric/Chat-bot-files/model.tflearn.chatbot


In [9]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    
    for se in s_words:
        for i, w, in enumerate(words):
            if w == se:
                bag[i] = (1)
    
    return numpy.array(bag)

def chat():
    print("start talking with the smithbot(type quit to exit): ")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break
        
        #now its time to feed the input into the model 
        
        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        print(tag)

In [10]:
chat()

start talking with the smithbot(type quit to exit): 
You: hello


ValueError: Cannot feed value of shape (1, 50) for Tensor 'InputData/X:0', which has shape '(?, 6)'